# Grids and Matrices

## Moving in a grid
Key approaches are: 
- factor out validation as a helper function, to avoid cluttering the main algorithm with checks.
- create a directions array that includes all possible changes to the current position to reach neighbouring cells.

#### Chess Moves
Create a function that returns possible moves for a king, queen or knight chess piece, given a board of size n x n, where 0 denotes an empty cell and 1 denotes an occupied one, and given starting coordinates.

In [77]:
# I factor out a validation factor, which checks if a given move would be within the confines of the board and to an unoccupied cell.
# This means I don't have to keep checking within the main algorithm.
def is_valid(board, r, c):
    n = len(board)
    if 0 <= r < n and 0 <= c < n:
        if board[r][c] == 0:
            return True
    return False


def chess_moves(board, piece, r, c):
    moves = []
    potential_moves = []

    if piece.lower() == "knight":
        # list comprehensions used to create directions arrays
        directions = [(rd, cd) for rd in [-2, 2] for cd in [-1, 1]] + [(rd, cd) for rd in [-1, 1] for cd in [-2, 2]]
    else: # king and queen both move in the same directions
        directions = [(rd, cd) for rd in [-1, 0, 1] for cd in [-1, 0, 1] if (rd != 0 or cd != 0)]  

    for rd, cd in directions:
        if piece.lower() == "queen": # As queen can travel across multiple spaces we need a loop to take her as far as 
            rm, cm = r+rd, c+cd      # she can go in each direction before hitting another piece or the end of the board.
            while is_valid(board, rm, cm):
                moves.append([rm, cm])
                rm+=rd
                cm+=cd
        else:
            if is_valid(board, r+rd, c+cd):
                moves.append([r+rd, c+cd])

    return moves

In [79]:
board = [[0, 0, 0, 1, 0, 0],
         [0, 1, 1, 1, 0, 0],
         [0, 1, 0, 1, 1, 0],
         [1, 1, 1, 1, 0, 0],
         [0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0]]

chess_moves(board, "king", 3, 5)

[[2, 5], [3, 4], [4, 4], [4, 5]]

#### Queen's reach
A board has a number of queens on it, indicated by ones. Return a board that uses zeros to denote spaces that are safe from the queens.

In [162]:
# One approach is to use the chess_moves algorithm to determine all of the potential moves for each queen. 
# This may be inefficient if there's a high concentration of queens. It may be better to go through each cell and check for threats.

def queens_reach(board):
    n = len(board)
    updated_board = [row.copy for row in board] # NB if you copy the grid, updating one will update the other. To create a new grid: [[0]*n for _ in range(n)]
    for r in range(n):
        for c in range(n):
            if board[r][c] == 1:
                updated_board[r][c] = 1
                moves = chess_moves(board, "queen", r, c)
                for rm, cm in moves:
                    updated_board[rm][cm] = 1
    return updated_board

In [164]:
board = [[0]*6 for _ in range(6)]
board[1][1] = 1
board[3][4] = 1
board[4][4] = 1
board

[[0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0]]

In [166]:
queens_reach(board)

[[1, 1, 1, 0, 1, 0],
 [1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1],
 [0, 1, 1, 1, 1, 1]]

##### Considerations
Each queen takes O(4n) so we get 4n x num_queens. The number of queens ranges from 0 to nxn; however, in the worst case, the queen will meet obstacles (other queens) on every side, so each queen would check only 8 cells. That makes the worst case O(8n^2) or O(n^2)

If we go cell by cell, each cell requires 4n checks, so the total is 4n x nxn = 4n^3. This approach would certainly be worse 
unless there are more queens than empty cells.

There may be another approach that would check for queens row by row, then column by column, then along the diagonals. Each dimension would be nxn so the total would be 4nxn. This would take half the time complexity of the worst case of the option coded, but more than the best case.

#### Spiral order
Given a +ve, odd integer, n, return an nxn grid, filled with every number from $0$ to $n^{2}-1$, spiralling clockwise down from the center.

In [284]:
# Validation function to check if coordinates are empty, should a turn be made
def turn(grid, r, c):
    return grid[r][c] == None

def spiral_order(n):
    grid = [[None]*n for _ in range(n)]
    dir = [[1,0], [0,-1], [-1,0], [0,1]] # Directions array. The function will need to cycle through these in order.
    d = -1 # d is a counter that determines which direction will be used.
    r, c = n//2, n//2
    for i in range(n**2):       
        grid[r][c] = i
        if turn(grid, r+dir[(d+1)%4][0], c+dir[(d+1)%4][1]): # Checks if making a turn is a valid move
            d = (d+1)%4 # Changes direction
        r += dir[d][0]
        c += dir[d][1]                   
    return grid    


In [286]:
spiral_order(5)

[[16, 17, 18, 19, 20],
 [15, 4, 5, 6, 21],
 [14, 3, 0, 7, 22],
 [13, 2, 1, 8, 23],
 [12, 11, 10, 9, 24]]

In [288]:
# The spiral always ends in the bottom right corner `grid[n][n]`.
# A conceptually easier approach is to start there and work backwards, using the edges of the grid to guide the movement.
def valid_move(grid, r, c):
    n = len(grid)
    if 0 <= r < n and 0 <= c < n:
        if grid[r][c] == 0:
            return True
    return False

def spiral_order_reverse(n):
    grid = [[0]*n for _ in range(n)]
    directions = [[-1,0],[0,-1],[1,0],[0,1]]
    d = 0
    r, c = n-1, n-1
    for i in range(n**2-1, 0, -1):
        grid[r][c] = i
        if not valid_move(grid, r+directions[d][0], c+directions[d][1]):
            d = (d + 1) % 4 # Changes direction
        r += directions[d][0]
        c += directions[d][1]
    return grid

In [290]:
spiral_order_reverse(5)

[[16, 17, 18, 19, 20],
 [15, 4, 5, 6, 21],
 [14, 3, 0, 7, 22],
 [13, 2, 1, 8, 23],
 [12, 11, 10, 9, 24]]

##### Analysis
Time complexity is $O(n^{2})$ for both models: the validation function takes constant time and is applied to each cell in the grid once. 
Spatial complexity is also $O(n^{2})$.

#### Snowprints
We are tracking a snow leopard crossing a rectangular binary grid where 1 denotes a snowprint. Movement is from left to right; each column must contain exactly one 1. Movement may shift up or down one row, or stay the same. What is the smallest number of rows between the path and top of the grid, where there's an icy river?

In [19]:
def is_occupied(grid, r, c):
    return grid[r][c] == 1

def snowprints(grid):
    proximity = 0
    # Find the footprint in the first row
    for r in range(len(grid)):
        if is_occupied(grid, r, 0):
            proximity = r
            break
    # We could follow the path, though it's only relevant if the leopard moves closer to the top. So we don't. We only move across or up.
    for c in range(1,len(grid[0])):
        if is_occupied(grid, proximity-1, c):
            proximity -= 1
            if proximity == 0: # We want to break if proximity shrinks to 0
                break
    return proximity

In [9]:
grid = [[0]*6 for _ in range(4)]
prints = [[2,0], [2,1], [1,2], [2,3], [3,4], [3,5]]
for r,c in prints:
    grid[r][c] = 1
grid

[[0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0],
 [1, 1, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 1]]

In [24]:
print(f"The snow leopard got as close as {snowprints(grid)} row(s) from the river.")

The snow leopard got as close as 1 row(s) from the river.


##### Analysis
While it may be possible that scanning the first few rows of the grid quickly turn up a result and that this is more efficient than the approach taken, I assume that the grid has more rows than columns and that the trade-off between knowing that the first column will contain a snowprint, and the risk that many rows may not, will work in favour of my approach. The worst possible time complexity is $O(n + m)$ where nxm are the dimensions of the grid. However, the break clauses will break the vertical and horizontal scans if, respectively, the first print is found early and the leopard reaches proximity 0. Compare this to a worst case of $O((n-1)*m)$. Space complexity is constant $O(1)$.

#### Valid sudoku
Given a 9x9 sudoku grid, return true if there are no conflicts. 0's denote empty cells.

In [63]:
def valid_subsection(arr):
    seen = set()
    for a in arr:
        if a in counts:
            return False
        if a != 0:
            seen.add(a)
    return True

def valid_sudoku(grid):
    # check the rows are valid
    for row in grid:
        if not valid_subsection(row):
            return False
    # now check the columns are valid
    for col in range(9):
        if not valid_subsection([grid[r][col] for r in range(9)]):
            return False
    # and finally, check the 3x3 subgrids are valid
    for i in range(3):
        for j in range(3):
            arr = [grid[i*3 + r][j*3 + c] for r in range(3) for c in range(3)]
            if not valid_subsection(arr):
                return False
    return True

In [41]:
#This will be used to create a populate a sudoku grid for testing purposes
def construct_grid(items):
    grid = [[0]* 9 for _ in range(9)]
    for r, c, i in items:
        grid[r][c] = i
    return grid

In [67]:
# Test on a valid example
items = [[0,0,5], [0,8,6], 
[1,2,9], [1,4,5], [1,6,3], 
[2,1,3],[2,5,2], 
[3,0,8],[3,3,7],[3,8,9],
[4,2,2],[4,6,8], 
[5,0,4],[5,5,6],[5,8,3], 
[6,3,3],[6,7,4],
[7,2,3],[7,4,8],[7,6,2],
[8,0,9],[8,8,7]]
grid = construct_grid(items)
valid_sudoku(grid)

True

In [71]:
# Now change one number to make the example invalid
grid[7][6] = 7
valid_sudoku(grid)

False

#### Subgrid maximums
Given an RxC grid of integers, return a new RxC grid where each cell gives the maximum of the subgrid formed spanning that cell and the bottom right corner.

In [55]:
def subgrid_maximums(grid):
    R = len(grid)
    C = len(grid[0])
    res = []
    
    def max_check(r, c):
        max = grid[r][c]
        for row in range(r,R):
            for col in range(c, C):
                if grid[row][col] > max:
                    max = grid[row][col]
        return max
    
    for r in range(R):
        res.append([])
        for c in range(C):
            res[r].append(max_check(r, c))
    return res 

In [65]:
grid = [[1, 5, 3],
        [4, -1, 0],
        [2, 0, 2]]

In [125]:
subgrid_maximums(grid)

[[5, 5, 3], [4, 2, 2], [2, 2, 2]]

The above approach inefficiently recalculates the maximum for every possible subgrid.  This takes time complexity $O((R*C)^2)$. A more efficent approach builds on existing maxima calculations.

In [123]:
def subgrid_maximums(grid):
    R = len(grid)
    C = len(grid[0])
    res = [[0] * C for _ in range(R)]

    for r in range(R-1, -1, -1):
        for c in range(C-1, -1, -1):
            current = grid[r][c]
            lower = res[r+1][c] if r < R-1 else current
            right = res[r][c+1] if c < C-1 else current
            res[r][c] = max(current, lower, right)
    return res    

#### Subgrid sums
Given an RxC grid of integers, return a new RxC grid where each cell gives the sum of the subgrid formed spanning that cell and the bottom right corner.

In [11]:
def subgrid_sums(grid):
    R, C = len(grid), len(grid[0])
    res = grid.copy()

    for r in range(R-1, -1, -1):
        for c in range(C-1, -1, -1):
            # curr += lower + right - lower-right
            res[r][c] += res[r+1][c] if r < R-1 else 0
            res[r][c] += res[r][c+1] if c < C-1 else 0
            res[r][c] -= res[r+1][c+1] if c< C-1 and r< R-1 else 0
    return res

In [15]:
grid = [[-1, 2, 3],
        [4, 0, 0],
        [-2, 0, 9]]
subgrid_sums(grid)

[[15, 14, 12], [11, 9, 9], [7, 9, 9]]